In [ ]:
!pip install langchain langchain-chroma langchain-community langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import numpy as np
import pandas as pd
import re
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/search_engine/subtitles_split_0.csv')

In [ ]:
df.head()

,num,name,content
0,9180533,the message (1976),"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here comes the grump s01 e09 joltin jack in bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis cells s02 e13 episode 2 13 (2022),"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis cells s02 e14 episode 2 14 (2022),"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker (2022),"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [ ]:
def cleaning_subtitles_text(text):
    lines = text.strip().split("\n")
    cleaned_lines = []
    for i in range(len(lines) - 1):
        if "-->" in lines[i]:
            line = lines[i + 1].strip().strip("-").strip()
            if 'subtitl' not in line.lower():  # Check for 'subtitle'
                cleaned_line = re.sub(r"[^\w\s]", "", line).lower()
                if cleaned_line:
                    cleaned_lines.append(cleaned_line)
    return "\n".join(cleaned_lines)

In [ ]:
df['content'] = df['content'].apply(cleaning_subtitles_text)

In [ ]:
df.head()

,num,name,content
0,9180533,the message (1976),in the name of god the most gracious the most ...
1,9180583,here comes the grump s01 e09 joltin jack in bo...,ah theres princess\nblooney looney soldiers\no...
2,9180592,yumis cells s02 e13 episode 2 13 (2022),iyumis cells 2i\niepisode 36\niyumii\nishould ...
3,9180594,yumis cells s02 e14 episode 2 14 (2022),iyumis cells 2i\niepisode 39 laptopi\nfirst pl...
4,9180600,broker (2022),if youre going to throw it away\nplease take c...


In [ ]:
def chunk_and_explode(df, text_column, chunk_size=850, chunk_overlap=50):
  """
  Chunks a dataframe based on a text column using RecursiveCharacterTextSplitter and explodes it.
  """

  # Initialize the text splitter
  splitter = RecursiveCharacterTextSplitter(chunk_size=850, chunk_overlap=60)

  df['chunks'] = df[text_column].apply(splitter.split_text)

  # Explode the dataframe based on the 'chunks' column
  exploded_df = df.explode('chunks')

  # Reset the index and return
  return exploded_df.reset_index(drop=True)

In [ ]:
exploded_df = chunk_and_explode(df, 'content')

In [ ]:
exploded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448047 entries, 0 to 448046
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   num      448047 non-null  int64 
 1   name     448047 non-null  object
 2   content  448047 non-null  object
 3   chunks   447122 non-null  object
dtypes: int64(1), object(3)
memory usage: 13.7+ MB


In [ ]:
df = exploded_df.drop(columns='content')

In [ ]:
df.tail()

,num,name,chunks
448042,9264094,charmed s01 e11 feats of clay (1999),then you risk paying the price\ni crashingi \n...
448043,9264094,charmed s01 e11 feats of clay (1999),i came for the thief\nwho are you\nim the guar...
448044,9264094,charmed s01 e11 feats of clay (1999),maybe this is something\ni mean it doesnt\nbut...
448045,9264094,charmed s01 e11 feats of clay (1999),right this way please\nthank you\ngood thing w...
448046,9264094,charmed s01 e11 feats of clay (1999),yeah\num\nyou know i hope\nill be the guy you\...


In [ ]:
def create_embeddings(df, text_column):
  embeddings_func = SentenceTransformerEmbeddings(
      model_name="sentence-transformers/all-MiniLM-L6-v2",
      model_kwargs = {'device': 'cuda'})

  embeddings = []
  for chunk in tqdm(df[text_column].values):
    if isinstance(chunk, str):
      embedding = embeddings_func.embed_query(chunk)
      embeddings.append(embedding)

    else:
      embeddings.append(None)

  df['chunk_embeddings'] = embeddings

  return df

In [ ]:
df_with_embeddings = create_embeddings(df, 'chunks')

100%|██████████| 448047/448047 [1:23:55<00:00, 88.97it/s]


In [ ]:
df_with_embeddings.to_csv('/content/drive/MyDrive/search_engine/processed_split_0.csv', index=False)